In [3]:
import numpy as np
import pandas as pd

# read in hh using list comprehension
hh = [line.rstrip('\n') for line in open('hh.txt')]
# display hh
hh

['Winning Poker Network Game #1081326940: No Limit Holdem (25/50) [2017/12/22 17:59:45 UTC]',
 'Table: $10 Regular 9-Max, Table 1',
 'Tournament: 8140214',
 'Seats: 9',
 'Seat 1: W1renut (1,439)',
 'Seat 2: Truck1536 (3,141)',
 'Seat 3: allegro88 (1,670)',
 'Seat 5: unclepete55 (1,721)',
 'Seat 6: Cutch123 (1,310)',
 'Seat 7: cobblestoner (1,201)',
 'Seat 8: otlichno (1,215)',
 'Seat 9: O.G. SKARFINGER (1,803)',
 'Button is seat 8',
 'O.G. SKARFINGER: posts small blind 25',
 'W1renut: posts big blind 50',
 '*** HOLE CARDS ***',
 'unclepete55: dealt [Qh Qs]',
 'Truck1536: folds',
 'allegro88: folds',
 'unclepete55: calls 50',
 'Cutch123: folds',
 'cobblestoner: folds',
 'otlichno: raises 200',
 'O.G. SKARFINGER: folds',
 'W1renut: raises 300',
 'unclepete55: folds',
 'otlichno: raises all-in 1,015',
 'W1renut: calls 865',
 '*** FLOP *** [Qc Ah 8s]',
 '*** TURN *** [Qc Ah 8s] [3h]',
 '*** RIVER *** [Qc Ah 8s] [3h] [2h]',
 '*** SUMMARY ***',
 'Pot: 2,505 | Rake: 0 |',
 'Board: [Qc Ah 8s 3

In [4]:
def get_stacks():
    # line.split()[1] is a string that i slice the first char with [:1]
    seats = [line.split()[1][:1] for line in hh if '(' in line]
    # convert seats to int and drop first and last items in seats list
    seats = [int(seat) for seat in seats[1:-1]]
    
    # get names associated with each seat
    names = [line.split()[2:-1] for line in hh if '(' in line]
    names = [name for name in names[1:-1]]
    
    # get stacks associated with each seat
    stacks = [line.split()[-1] for line in hh if '(' in line]
    # drop parenthesis on each stack and drop the comma and convert stack to int
    # note: dropping first and last items in list as in seats
    stacks = [int(stack[1:-1].replace(',', '')) for stack in stacks[1:-1]]
    
    # identify which seat is the button
    # note this list comprehension only returns a one element list which I extract
    btn = [int(line.split()[-1]) for line in hh if 'Button is seat ' in line][0]
    
    index_btn = seats.index(btn)
    
    # create position list in appropriate deal order for 9 players PT4 convention
    pos = [6,5,4,3,2,1,0,9,8]
    pos = pos[9-len(stacks):]
    num_stacks = len(stacks)
    # repeat list to create a window
    stacks1 = (stacks * 2)[index_btn:index_btn+num_stacks]
    names1 = (names * 2)[index_btn:index_btn+num_stacks]
    # join stacks to match deal order and positions
    stacks2 = stacks1[3:] + stacks1[:3]
    names2 = names1[3:] + names1[:3]
    df = pd.DataFrame({'pos': pos, 'name': names2, 'chips': stacks2}).set_index('pos')
    
    return df

get_stacks()


,name,chips
pos,,
5,[Truck1536],3141
4,[allegro88],1670
3,[unclepete55],1721
2,[Cutch123],1310
1,[cobblestoner],1201
0,[otlichno],1215
9,"[O.G., SKARFINGER]",1803
8,[W1renut],1439


In [5]:
def get_bb():
    bb = [int(line.split()[-1]) for line in hh if 'posts big blind' in line][0]
    return bb

get_bb()

50

In [6]:
def get_sb():
    sb = [int(line.split()[-1]) for line in hh if 'posts small blind' in line][0]
    return sb

get_sb()

25

In [47]:
# for the purpose of my study I am calculating 9-man sng payout structure
# 0.5, 0.3, 0.2 to 1st, 2nd, 3rd respectively
def ICM9(df):
    payouts = [0.5, 0.3, 0.2]
    tot = df.chips.sum()
    length = len(df.chips)
    # df.left1 is the amount of chips remaining if that player wins 1st place
    df['left1'] = tot - df.chips
    
    df['p1st'] = df.chips / tot
    
    
    # left2 is a df of chips remaining if players i and j get 1st and 2nd
    # note that who gets first and who gets second is irrelevant since the chips remaining
    #      will be the same
    left2 = pd.DataFrame(np.nan, index = range(length), columns = range(len(df.chips)))
    for i in range(len(df.chips)-1):
        for j in range(i+1,len(df.chips)):
            left2.iloc[i,j] = tot - df.chips.iloc[i] - df.chips.iloc[j]
    
    
    p2 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
    # i is the row and represents the player placing first in the tournament
    for i in range(length):
        # j is the player's equity in 2nd place given player i is the winner
        for j in range(length):
            # if i == j then player cannot be second because she has already won
            if i != j:
                p2.iloc[i,j] = df.p1st.iloc[i] * df.chips.iloc[j] / df.left1.iloc[i] 
                
    df['eq1st'] = df.p1st * payouts[0]
    p2nd = [p2[i].sum() for i in range(length)]
    df['p2nd'] = p2nd
    df['eq2nd'] = df.p2nd * payouts[1]
    
    # multiply p2 and chips[i] / left2
    
    p3_list = []
    for ind, stack in enumerate(df.chips):
        p3 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
        for i in range(length):
            for j in range(i+1, length):
                if (i != ind) & (j != ind):
                    p3.iloc[i,j] = stack / left2.iloc[i,j] * p2.iloc[i,j]
                    p3.iloc[j,i] = stack / left2.iloc[i,j] * p2.iloc[j,i]
        
        p3_list.append(p3.sum().sum())
    
    df['p3rd'] = p3_list
    df['eq3rd'] = df.p3rd * payouts[2]
    df['$EV'] = df.eq1st + df.eq2nd + df.eq3rd 
    return df

res = ICM9(get_stacks())
res




,name,chips,left1,p1st,eq1st,p2nd,eq2nd,p3rd,eq3rd,$EV
pos,,,,,,,,,,
5,[Truck1536],3141,10359,0.232667,0.116333,0.201197,0.060359,0.169884,0.033977,0.210669
4,[allegro88],1670,11830,0.123704,0.061852,0.127018,0.038105,0.129670,0.025934,0.125891
3,[unclepete55],1721,11779,0.127481,0.063741,0.130267,0.039080,0.132185,0.026437,0.129258
2,[Cutch123],1310,12190,0.097037,0.048519,0.102907,0.030872,0.109385,0.021877,0.101268
1,[cobblestoner],1201,12299,0.088963,0.044481,0.095218,0.028565,0.102375,0.020475,0.093522
0,[otlichno],1215,12285,0.090000,0.045000,0.096215,0.028865,0.103298,0.020660,0.094524
9,"[O.G., SKARFINGER]",1803,11697,0.133556,0.066778,0.135401,0.040620,0.136042,0.027208,0.134606
8,[W1renut],1439,12061,0.106593,0.053296,0.111778,0.033533,0.117162,0.023432,0.110262
